### __Prepare__

> 에브리타임은 원칙상 크롤링을 자유롭게 할 수 없는 사이트므로 차단 방지 작업 필요

__1. robots.txt 확인__
* https://everytime.kr/robots.txt
* User-agent: 허용하는 검색봇 이름
* Allow: 허용하는 경로
* Disallow: 접근 불가능한 경로

__2. code 내의 작업__
* User-Agents 설정하기(header에 넣어줄 부분)
* sleep으로 페이지마다 쉬어주기(time.sleep)
* 동일한 시간이 아니라 랜덤하게 바꾸어주기(random.uniform(3,5))
* 반복 횟수가 많을수록 더 많이 쉬어주기

### __1. Lecture Crawling__

> 에브리타임 시간표를 이용한 서울시립대학교 강의목록 크롤링 

__1. 강의정보__
* 학년
* 교과번호
* 교과목명
* 학점
* 교수
* 강의시간
* 강의유형
* 담은인원
* 정원

__2. 리뷰정보__
* 강의평
* 링크

In [2]:
#라이브러리 로드
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
from tqdm.notebook import tqdm
import re
from time import sleep
import time
import warnings
import random
from openpyxl import Workbook,load_workbook
warnings.filterwarnings(action='ignore')

In [ ]:
#드라이버 정의
url = 'https://everytime.kr/timetable'
driver = webdriver.Chrome('/Users/diekim/downloads/chromedriver')
driver.get(url)
driver.maximize_window()
time.sleep(random.uniform(3,5))

In [ ]:
#에브리타임 로그인   
id_var = input('에브리타임 아이디를 입력하세요: ')
pw_var = input('에브리타임 비밀번호를 입력하세요: ')
driver.find_element_by_name('userid').send_keys(id_var)
driver.find_element_by_name('password').send_keys(pw_var)
driver.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()
time.sleep(random.uniform(3,5))

In [ ]:
#강의목록 스크롤
driver.find_element_by_xpath('//*[@id="container"]/ul/li[1]').click()
time.sleep(random.uniform(3,5))

pre_cnt = 0 
while True: #끝까지 내리기
    #현재 페이지 요소 접근 
    element = driver.find_elements_by_css_selector("#subjects > div.list > table > tbody > tr")
    time.sleep(random.uniform(3,5))
    #현재 페이지 끝까지 스크롤
    driver.execute_script('arguments[0].scrollIntoView(true);', element[-1])
    time.sleep(random.uniform(3,5))
    #같으면 루프 탈출 
    current_cnt = len(element)
    if pre_cnt == current_cnt:
        break
    #같아질 때까지 반복
    pre_cnt = current_cnt
    time.sleep(random.uniform(3,5))

In [ ]:
#강의목록 크롤링  
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
trs = soup.select('#subjects > div.list > table > tbody > tr')
results = []
for tr in trs:
    result=[]
    tds = tr.select('#subjects > div.list > table > tbody > tr > td')
    score = tr.select('#subjects > div.list > table > tbody > tr > td')[8].a['title'] #강의평
    link = tr.select('#subjects > div.list > table > tbody > tr > td')[8].a['href']  #링크
    result.append(tds[0].text)  #과목코드
    result.append(tds[1].text)  #과목명
    result.append(tds[2].text)  #교수
    result.append(tds[3].text)  #강의시간
    result.append(tds[4].text)  #강의실
    result.append(tds[5].text)  #구분
    result.append(tds[6].text)  #학년
    result.append(tds[7].text)  #학점
    #result.append(tds[8].text) #강의평
    result.append(tds[9].text)  #담은인원
    result.append(tds[10].text) #정원
    result.append(tds[11].text) #수강대상
    result.append(tds[12].text) #비고
    result.append(score)        #강의평
    result.append(link)         #링크
    results.append(result)

In [ ]:
#강의목록 저장
columns = ['과목코드', '과목명', '교수', '강의시간', '강의실', '구분', '학년', 
           '학점', '담은인원', '정원', '수강대상', '비고', '강의평', '링크']
columnn = 14
write_wb = Workbook()
write_ws = write_wb.active
write_ws.append(columns)
for data in results:
    write_ws.append(data)
write_wb.save('강의목록.xlsx')
df = pd.read_excel("강의목록.xlsx")
df.to_csv('강의목록.csv', encoding='utf-8-sig', index=False)

In [ ]:
#최종 크롤러 정의
def lecture_cralwer():
    
    #라이브러리 로드
    import pandas as pd
    import numpy as np
    from bs4 import BeautifulSoup
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from tqdm import tqdm
    from tqdm.notebook import tqdm
    import re
    from time import sleep
    import time
    import warnings
    import random
    from openpyxl import Workbook,load_workbook
    warnings.filterwarnings(action='ignore')
    
    #드라이버 정의
    url = 'https://everytime.kr/timetable'
    driver = webdriver.Chrome('/Users/diekim/downloads/chromedriver')
    driver.get(url)
    driver.maximize_window()
    time.sleep(random.uniform(3,5))
    
    #에브리타임 로그인   
    id_var = input('에브리타임 아이디를 입력하세요: ')
    pw_var = input('에브리타임 비밀번호를 입력하세요: ')
    driver.find_element_by_name('userid').send_keys(id_var)
    driver.find_element_by_name('password').send_keys(pw_var)
    driver.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()
    time.sleep(random.uniform(3,5))
    
    #강의목록 스크롤
    driver.find_element_by_xpath('//*[@id="container"]/ul/li[1]').click()
    time.sleep(random.uniform(3,5))

    pre_cnt = 0 
    while True: #끝까지 내리기
        #현재 페이지 요소 접근 
        element = driver.find_elements_by_css_selector("#subjects > div.list > table > tbody > tr")
        time.sleep(random.uniform(3,5))
        #현재 페이지 끝까지 스크롤
        driver.execute_script('arguments[0].scrollIntoView(true);', element[-1])
        time.sleep(random.uniform(3,5))
        #같으면 루프 탈출 
        current_cnt = len(element)
        if pre_cnt == current_cnt:
            break
        #같아질 때까지 반복
        pre_cnt = current_cnt
        time.sleep(random.uniform(3,5))
        
    #강의목록 크롤링
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    trs = soup.select('#subjects > div.list > table > tbody > tr')
    results = []
    for tr in trs:
        result=[]
        tds = tr.select('#subjects > div.list > table > tbody > tr > td')
        score = tr.select('#subjects > div.list > table > tbody > tr > td')[7].a['title'] 
        link = tr.select('#subjects > div.list > table > tbody > tr > td')[7].a['href']   
        #학교마다 구성 다름 
        result.append(tds[0].text)  #학년
        result.append(tds[1].text)  #교과번호-분반
        result.append(tds[2].text)  #교과목명
        result.append(tds[3].text)  #학점
        result.append(tds[4].text)  #교수
        result.append(tds[5].text)  #강의시간/강의실
        result.append(tds[6].text)  #강의유형
        #result.append(tds[7].text) #강의평
        result.append(tds[8].text)  #담은인원
        result.append(tds[9].text)  #정원
        result.append(score)        #강의평
        result.append(link)         #링크
        results.append(result)
        
    #강의목록 저장
    columns = ['학년', '교과번호', '교과목명', '학점', '교수' ,'강의시간',
               '강의유형', '담은인원', '정원' ,'강의평', '링크']
    columnn = 11
    write_wb = Workbook()
    write_ws = write_wb.active
    write_ws.append(columns)
    for data in results:
        write_ws.append(data)
    write_wb.save('강의목록.xlsx')
    df = pd.read_excel("강의목록.xlsx")
    df.to_csv('강의목록.csv', encoding='utf-8-sig', index=False)
    
    driver.quit()
    return df

In [ ]:
#강의목록 크롤링
df = lecture_cralwer()
df

,학년,교과번호,교과목명,학점,교수,강의시간,강의유형,담은인원,정원,강의평,링크
0,1.0,01569-01,영문분석과활용,3,김선희,"금02,03,04/4-204~6",교양선택,35,25,4.83,/lecture/view/2090902
1,1.0,01570-01,영어면접과발표,3,William Hart,"수05/4-208~10, 목03,04/4-208~10",교양선택,26,28,4.67,/lecture/view/2090903
2,1.0,01573-01,영어말하기와토론,3,Joseph Van Dorn,"월06,07,08/20-320",교양선택,17,25,0.00,/lecture/view/2379676
3,1.0,01573-02,영어말하기와토론,3,Steven Moore,"수05/4-204~6, 목03,04/4-204~6",교양선택,26,28,4.54,/lecture/view/2090905
4,1.0,01668-01,시사영어,3,김선희,"금07,08,09/4-204~6",교양선택,27,25,4.57,/lecture/view/2090906
...,...,...,...,...,...,...,...,...,...,...,...
2802,NaN,I4900-02,논문연구,0,박병은,NaN,전공선택,0,1,0.00,/lecture/view/1376285
2803,NaN,521-02,윤강,2,한인섭,"목10,11/4-432",전공선택,0,6,0.00,/lecture/view/2380242
2804,0.0,40.900-01,논문연구,0,이수일,NaN,전공선택,0,0,0.00,/lecture/view/296156
2805,0.0,49.900-02,논문연구,0,김한준,NaN,전공선택,0,1,0.00,/lecture/view/296191


In [ ]:
#전체 강의 중 교양선택 과목에 대해서 진행할 예정
df = df[df['강의유형']=='교양선택']
df.to_csv('강의목록_교양선택.csv', encoding='utf-8-sig', index=False)

### __2. Review Crawling__

> 교양과목 추천을 위해서 구분이 교양선택인 과목에 대해 강의리뷰 크롤링

__1. 공통 정보__
* 강의명
* 교수명
* 개설학기
* 평균평점
* 과제
* 조모임
* 성적
* 출결
* 시험

__2. 개별 정보__
* 유저평점
* 수강학기
* 리뷰내용

In [ ]:
#라이브러리 로드
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
from tqdm.notebook import tqdm
import re
from time import sleep
import time
import warnings
import random
from openpyxl import Workbook,load_workbook
warnings.filterwarnings(action='ignore')

#드라이버 정의
url = url = 'https://everytime.kr/' + 'lecture/view/2198679'
driver = webdriver.Chrome('/Users/diekim/downloads/chromedriver')
driver.get(url)
driver.maximize_window()
time.sleep(random.uniform(3,5))

#에브리타임 로그인   
id_var = input('에브리타임 아이디를 입력하세요: ')
pw_var = input('에브리타임 비밀번호를 입력하세요: ')
driver.find_element_by_name('userid').send_keys(id_var)
driver.find_element_by_name('password').send_keys(pw_var)
driver.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()
time.sleep(random.uniform(3,5))

In [ ]:
#공통정보
x1 = driver.find_element_by_xpath('/html/body/div[1]/div[2]/h2').text #강의명
x2 = driver.find_element_by_xpath('/html/body/div[1]/div[2]/p[1]/span').text #교수명
x3 = driver.find_element_by_xpath('/html/body/div[1]/div[2]/p[2]/span').text #개설학기
try:
    x4 = driver.find_element_by_xpath('/html/body/div[1]/div[4]/div[1]/div[1]/span/span[1]').text #평균평점
    x5 = driver.find_element_by_xpath('/html/body/div[1]/div[4]/div[1]/div[2]/p[1]/span').text #과제
    x6 = driver.find_elements_by_xpath('/html/body/div[1]/div[4]/div[1]/div[2]/p[2]/span')[0].text #조모임
    x7 = driver.find_elements_by_xpath('/html/body/div[1]/div[4]/div[1]/div[2]/p[3]/span')[0].text #성적
    x8 = driver.find_elements_by_xpath('/html/body/div[1]/div[4]/div[1]/div[2]/p[4]/span')[0].text #출결
    x9 = driver.find_elements_by_xpath('/html/body/div[1]/div[4]/div[1]/div[2]/p[5]/span')[0].text #시험
except:
    pass

print(x1, x2, x3, x4, x5, x6, x7, x8, x9)

In [ ]:
#개별정보
i=0; x10=[]; x11=[]; x12=[]
while True:
    i += 1
    try:
        #유저평점
        x10.append(driver.find_elements_by_xpath(
            '/html/body/div[1]/div[4]/div[2]/article['+str(i)+']/p[1]/span/span'
        )[0].get_attribute('style'))
        #수강학기
        x11.append(driver.find_elements_by_xpath(
            '/html/body/div[1]/div[4]/div[2]/article['+str(i)+']/p[2]/span'
        )[0].text) 
        #리뷰내용
        x12.append(driver.find_elements_by_xpath(
            '/html/body/div[1]/div[4]/div[2]/article['+str(i)+']/p[3]'
        )[0].text)
    except:
        break

print(x10, x11, x12)                

['width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 40%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;', 'width: 100%;'] ['21년 2학기 수강자', '21년 1학기 수강자', '21년 2학기 수강자', '21년 1학기 수강자', '21년 2학기 수강자', '21년 2학기 수강자', '21년 2학기 수강자', '21년 2학기 수강자', '21년 2학기 수강자', '21년 2학기 수강자', '21년 2학기 수강자', '21년 2학기 수강자', '21년 2학기 수강자', '21년 1학기 수강자', '21년 1학기 수강자', '21년 1학기 수강자', '21년 1학기 수강자', '21년 1학기 수강자', '21년 1학기 수강자', '21년 1학기 수강자', '21년 1학기 수강자', '21년 1학기 수강자', '21년 1학기 수강자', '21년 1학기 수강자', '21년 1학기 수강자', '21년 1학기 수강자'] ['꿀강이라는 말 듣고 수강했었는데 강의하나도 안봤는데 시험 다 찝어주셔서 시험 전 주에 공부하고 a+받았습니다ㅠ 레포트는 2개있었고, 시험은 객관식 주관식 서술형있었는데 거기에 주관식은 보기까지 있었어요!! 진짜 밥상차려준 과목...\n교수님도 한글자라도 더 쓰면 점수준다고 말씀하시는 꿀강입니다!!', '꿀교양 of 꿀교양\n시험보다 수강신청

In [ ]:
#최종 크롤러 정의
def review_cralwer(URL):
    
    #라이브러리 로드
    import pandas as pd
    import numpy as np
    from bs4 import BeautifulSoup
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from tqdm import tqdm
    from tqdm.notebook import tqdm
    import re
    from time import sleep
    import time
    import warnings
    import random
    from openpyxl import Workbook,load_workbook
    warnings.filterwarnings(action='ignore')
        
    #전체 강의 목록에 대해서 반복
    res = pd.DataFrame()
    driver = webdriver.Chrome('/Users/diekim/downloads/chromedriver')
    for k, url in enumerate(URL): 
        print('{}번째 강의의 리뷰데이터를 수집합니다...'.format(k+1))
        url = 'https://everytime.kr/' + url
        
        #드라이버 정의
        driver.get(url)
        driver.maximize_window()
        time.sleep(random.uniform(3,5))
        
        #에브리타임 로그인   
        if k==0:
            id_var = input('에브리타임 아이디를 입력하세요: ')
            pw_var = input('에브리타임 비밀번호를 입력하세요: ')
            driver.find_element_by_name('userid').send_keys(id_var)
            driver.find_element_by_name('password').send_keys(pw_var)
            driver.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()
            time.sleep(random.uniform(3,7))
        else:
            pass

        #공통정보
        x1=[];x2=[];x3=[];x4=[];x5=[];x6=[];x7=[];x8=[];x9=[]
        x1.append(driver.find_element_by_xpath('/html/body/div[1]/div[2]/h2').text) #강의명
        try:
            x2.append(driver.find_element_by_xpath('/html/body/div[1]/div[2]/p[1]/span').text) #교수명
        except:
            pass
        try:   
            x3.append(driver.find_element_by_xpath('/html/body/div[1]/div[2]/p[2]/span').text) #개설학기
        except:
            pass
        try:
            x4.append(driver.find_element_by_xpath('/html/body/div[1]/div[4]/div[1]/div[1]/span/span[1]').text)  #평균평점
            x5.append(driver.find_element_by_xpath('/html/body/div[1]/div[4]/div[1]/div[2]/p[1]/span').text)     #과제
            x6.append(driver.find_elements_by_xpath('/html/body/div[1]/div[4]/div[1]/div[2]/p[2]/span')[0].text) #조모임
            x7.append(driver.find_elements_by_xpath('/html/body/div[1]/div[4]/div[1]/div[2]/p[3]/span')[0].text) #성적
            x8.append(driver.find_elements_by_xpath('/html/body/div[1]/div[4]/div[1]/div[2]/p[4]/span')[0].text) #출결
            x9.append(driver.find_elements_by_xpath('/html/body/div[1]/div[4]/div[1]/div[2]/p[5]/span')[0].text) #시험
        except:
            pass
        time.sleep(random.uniform(3,5))
        
        #개별정보
        i=0; x10=[]; x11=[]; x12=[]
        while True:
            i += 1
            print('{}번째 리뷰 크롤링 중...'.format(i))
            time.sleep(random.uniform(3,5))
            try:
                #유저평점
                x10.append(driver.find_elements_by_xpath(
                    '/html/body/div[1]/div[4]/div[2]/article['+str(i)+']/p[1]/span/span'
                )[0].get_attribute('style'))
                #수강학기
                x11.append(driver.find_elements_by_xpath(
                    '/html/body/div[1]/div[4]/div[2]/article['+str(i)+']/p[2]/span'
                )[0].text) 
                #리뷰내용
                x12.append(driver.find_elements_by_xpath(
                    '/html/body/div[1]/div[4]/div[2]/article['+str(i)+']/p[3]'
                )[0].text)
            except:
                break              

        #데이터프레임 저장
        tmp = pd.DataFrame()
        tmp['강의명'] = x1*(i-1)
        tmp['교수명'] = x2*(i-1)
        tmp['개설학기'] = x3*(i-1)
        tmp['평균평점'] = x4*(i-1)
        tmp['과제'] = x5*(i-1)
        tmp['조모임'] = x6*(i-1)
        tmp['성적'] = x7*(i-1)
        tmp['출결'] = x8*(i-1)
        tmp['시험'] = x9*(i-1)
        tmp['유저평점'] = x10
        tmp['수강학기'] = x11
        tmp['리뷰내용'] = x12
        res = pd.concat([res, tmp])
        time.sleep(random.uniform(3,5))
    
    driver.quit()
    return res

In [ ]:
#강의리뷰 크롤링
URL = pd.read_csv('강의목록_교양선택.csv').iloc[:, -1]
URL = URL.drop_duplicates()
df = review_cralwer(URL)
df.to_csv('강의리뷰_교양선택.csv', encoding='utf-8-sig', index=False)
df

### __3. Data Preprocessing__
> 강의목록 데이터와 강의리뷰 데이터의 병합 및 데이터 전처리 진행

__1. 데이터 병합__
* 강의목록_교양선택.csv
* 강의리뷰_교양선택.csv

__2. 데이터 전처리__
* 유저평점 칼럼 수정
* 리뷰수 칼럼 추가
* 시험 칼럼 수정

In [ ]:
#데이터 불러오기
import pandas as pd
pd.set_option('display.max_columns', None)
df1 = pd.read_csv('강의목록_교양선택.csv')
df2 = pd.read_csv('강의리뷰_교양선택.csv')
display(df1.head(2), df1.shape)
display(df2.head(2), df2.shape)

,학년,교과번호,교과목명,학점,교수,강의시간,강의유형,담은인원,정원,강의평,링크
0,1.0,01569-01,영문분석과활용,3,김선희,"금02,03,04/4-204~6",교양선택,35,25,4.83,/lecture/view/2090902
1,1.0,01570-01,영어면접과발표,3,William Hart,"수05/4-208~10, 목03,04/4-208~10",교양선택,26,28,4.67,/lecture/view/2090903


(265, 11)

,강의명,교수명,개설학기,평균평점,과제,조모임,성적,출결,시험,유저평점,수강학기,리뷰내용
0,영문분석과활용,김선희,"2022-1, 2021-1, 2020-1",4.83,보통,보통,너그러움,직접호명,두 번,width: 100%;,20년 1학기 수강자,영어 잘하시면 성적 잘받을수 있습니당 ㅎㅎ (토플라이팅 29점 기준!)
1,영문분석과활용,김선희,"2022-1, 2021-1, 2020-1",4.83,보통,보통,너그러움,직접호명,두 번,width: 80%;,21년 1학기 수강자,교수님 강의력은 정말 뛰어나시고 질문도 잘 받아주심.\n그런데 매주 문제 풀어오는 ...


(7186, 12)

In [ ]:
#교과목명 기준 merge
df1 = df1.drop_duplicates(subset=['링크'])
df = pd.merge(df1, df2, how='inner', left_on=['교과목명', '교수'], right_on=['강의명', '교수명'])
df = df[['강의명', '교수명', '학년', '교과번호', '학점', '강의시간', '강의유형', '담은인원', '정원', '링크',
         '개설학기' ,'평균평점', '과제', '조모임', '성적', '출결', '시험', '유저평점', '수강학기', '리뷰내용']]
display(df.head(2), df.shape)

,강의명,교수명,학년,교과번호,학점,강의시간,강의유형,담은인원,정원,링크,개설학기,평균평점,과제,조모임,성적,출결,시험,유저평점,수강학기,리뷰내용
0,영문분석과활용,김선희,1.0,01569-01,3,"금02,03,04/4-204~6",교양선택,35,25,/lecture/view/2090902,"2022-1, 2021-1, 2020-1",4.83,보통,보통,너그러움,직접호명,두 번,width: 100%;,20년 1학기 수강자,영어 잘하시면 성적 잘받을수 있습니당 ㅎㅎ (토플라이팅 29점 기준!)
1,영문분석과활용,김선희,1.0,01569-01,3,"금02,03,04/4-204~6",교양선택,35,25,/lecture/view/2090902,"2022-1, 2021-1, 2020-1",4.83,보통,보통,너그러움,직접호명,두 번,width: 80%;,21년 1학기 수강자,교수님 강의력은 정말 뛰어나시고 질문도 잘 받아주심.\n그런데 매주 문제 풀어오는 ...


(7186, 20)

In [ ]:
#결측값, 중복값 없음
print(df.isnull().sum())
print(df.duplicated().sum())

강의명     0
교수명     0
학년      0
교과번호    0
학점      0
강의시간    0
강의유형    0
담은인원    0
정원      0
링크      0
개설학기    0
평균평점    0
과제      0
조모임     0
성적      0
출결      0
시험      0
유저평점    0
수강학기    0
리뷰내용    0
dtype: int64
0


In [ ]:
#유저평점 칼럼 수정
print(df['유저평점'].value_counts())

def for_user_rating(x):
    if x=='width: 100%;':
        return 5.
    elif x=='width: 80%;':
        return 4.
    elif x=='width: 60%;':
        return 3. 
    elif x=='width: 40%;':
        return 2.
    else:
        return 1.
    
df['유저평점'] = df['유저평점'].apply(for_user_rating)
df[['유저평점']]

width: 100%;    3625
width: 80%;     1782
width: 60%;     1110
width: 20%;      366
width: 40%;      303
Name: 유저평점, dtype: int64


,유저평점
0,5.0
1,4.0
2,5.0
3,5.0
4,5.0
...,...
7181,4.0
7182,3.0
7183,1.0
7184,4.0


In [ ]:
#리뷰수 칼럼 추가
df['리뷰수'] = df.groupby('링크').transform('count').iloc[:, -1]
df[['리뷰수']]

,리뷰수
0,6
1,6
2,6
3,6
4,6
...,...
7181,24
7182,24
7183,24
7184,24


In [ ]:
#시험 칼럼 수정
print(df['시험'].value_counts())

def for_test(x):
    if x=='없음':
        return 0
    elif x=='한 번':
        return 1
    elif x=='두 번':
        return 2
    elif x=='세 번':
        return 3
    else:
        return 4

df['시험'] = df['시험'].apply(for_test)
df[['시험']]

두 번       5072
한 번        928
없음         846
네 번 이상     328
세 번         12
Name: 시험, dtype: int64


,시험
0,2
1,2
2,2
3,2
4,2
...,...
7181,0
7182,0
7183,0
7184,0


In [ ]:
#최종 데이터 저장
df.to_csv('에브리타임_크롤링_최종.csv', encoding='utf-8-sig', index=False)

### __Reference__
* https://velog.io/@psr2110/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC-%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B81%EC%A3%BC%EC%B0%A8-%ED%81%AC%EB%A1%A4%EB%A7%81
* https://github.com/wwlee94/everytime-timetable-crawling
* http://www.datamarket.kr/xe/index.php?mid=board_BoGi29&document_srl=63034&listStyle=viewer